In [13]:
import nibabel as nib
from nilearn import plotting, image
import numpy as np

def calcular_sobreposicao(mascara1, mascara2):
    sobreposicao = np.sum(mascara1 * mascara2) / np.sum(np.logical_or(mascara1, mascara2))
    return sobreposicao

# Carregar submáscaras da máscara 1 a partir de arquivos .nii
arquivo_nii_submascara1 = r'D:\ERICK\MD\ROIs for analysis\rBA1.nii'
arquivo_nii_submascara2 = r'D:\ERICK\MD\ROIs for analysis\rBA2.nii'
arquivo_nii_submascara3 = r'D:\ERICK\MD\ROIs for analysis\rBA3.nii'

submascara1 = nib.load(arquivo_nii_submascara1)
submascara2 = nib.load(arquivo_nii_submascara2)
submascara3 = nib.load(arquivo_nii_submascara3)
# Somar os valores das submáscaras
mascara1 = submascara1.get_fdata() + submascara2.get_fdata() + submascara3.get_fdata()
mascara1 = np.where(mascara1 != 0, 1, 0)

# Carregar máscara 2 (ito)
arquivo_nii2 = r'D:\ERICK\MD\ROIs for analysis\rNet2.nii'
mascara2 = nib.load(arquivo_nii2)
mascara2 = mascara2.get_fdata()

# Carregar máscara 3 (ICA)
arquivo_nii3= r'D:\ERICK\MD\ROIs for analysis\component2_mask.nii'
mascara3 = nib.load(arquivo_nii3)
mascara3 = mascara3.get_fdata()[:, :, :, 0]

# Calcular sobreposição
sobreposicao = calcular_sobreposicao(mascara1, mascara2)

# print(f"Soma das submáscaras: {np.sum(mascara1)}")
print(f"Sobreposição: {sobreposicao}")

Sobreposição: 0.07034822370735139


In [3]:
# sobreposição entre máscaras Glasser
arquivo_nii1 = r'D:\ERICK\MD\ROIs for analysis\rNet1.nii' #visual
mascara1 = nib.load(arquivo_nii1)
mascara1 = mascara1.get_fdata()

arquivo_nii2 = r'D:\ERICK\MD\ROIs for analysis\rNet11.nii' #dan
mascara2 = nib.load(arquivo_nii2)
mascara2 = mascara2.get_fdata()

arquivo_nii3 = r'D:\ERICK\MD\ROIs for analysis\rNet3.nii' #cing
mascara3 = nib.load(arquivo_nii3)
mascara3 = mascara3.get_fdata()

# Calcular sobreposição
sobreposicao = calcular_sobreposicao(mascara1, mascara2)

# print(f"Soma das submáscaras: {np.sum(mascara1)}")
print(f"Sobreposição: {sobreposicao}")

Sobreposição: 0.0


In [16]:
#plotando mapa cerebral
sobrep = mascara1 + mascara2*2 #+ mascara3*3
affine = submascara1.affine
mapa = nib.Nifti1Image(sobrep, affine)
view = plotting.view_img(mapa, cmap='jet', cut_coords=(0, 0, 0), colorbar=False, title='Sobreposição das máscaras')
view.open_in_browser()

C:\Users\erick\.conda\envs\erickenv\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [14]:
# visualização glasser
arquivo_nii = r'D:\ERICK\MD\ROIs for analysis\rNet1.nii'
mapa = nib.load(arquivo_nii)
mapa = nib.Nifti1Image(mascara.get_fdata(), affine)
view = plotting.view_img(mapa, cmap='jet', cut_coords=(0, 0, 0), colorbar=True, title='Mapa Cerebral')
view.open_in_browser()

C:\Users\erick\.conda\envs\erickenv\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
C:\Users\erick\.conda\envs\erickenv\Lib\site-packages\nilearn\image\resampling.py:291: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(


In [1]:
# compactação das imagens 4D
import ants
import os
import os.path as op
import glob
import pandas as pd
import numpy as np

first_subs = ['04', '05', '06', '07']
subs = list(np.arange(10, 46, 1))
subs = [str(subj) for subj in subs]
subs = first_subs + subs

excluded = [8, 9, 41, 45]
tr = 0.8
overwrite = True
mode = 'full'
df = pd.read_csv(r"C:\Users\erick\Documents\mestrado\master_dataset.csv")

root = r"C:\Users\erick\Documents\mestrado\subjects"

for sub in subs:
    
    if int(sub) in excluded:
        print('skipping subject:', sub)
        continue
        
    os.chdir(root)
    j = int(sub)
    sub_dir = glob.glob("{}_CISC*".format(sub))[0]
    t_corr = np.array(pd.read_csv(f'D:/ERICK/MD/extracted_voxels/{int(sub)}/time_corr.csv', header=None))
    
    # Definir o diretório de entrada
    if op.isdir(sub_dir):
        print('-----> ', sub_dir)
        input_dir = root + '\\{}'.format(sub_dir)
        
        os.chdir(input_dir)
        blocks = glob.glob("fMRI_block_*")

        # Carregar os EPIs
        for i, block_folder in enumerate(blocks):
            epis = []
            irun = int(block_folder[11])
            print('run:', irun)
            block_dir = input_dir + '\\' + block_folder
            # Definir o nome do arquivo de saída
            output_file = root + '\\task_volumes' + "\\subject_{}_run_{}_{}.nii.gz".format(sub, irun, mode)
            
            if not op.isfile(output_file) or overwrite:
            
                run_data = df.query('rawSubjID == @j & run == @irun')
                onsets = np.array(run_data['movieStartSecs'])
                onsets_TR = []
                offsets = np.array(onsets + run_data['veridicalDuration'])
                offsets_TR = []

                for itrial in range(len(offsets)):
                    onsets_TR.append(np.floor(onsets[itrial]/tr) - t_corr[i, itrial])
                    offsets_TR.append(np.floor(offsets[itrial]/tr) - t_corr[i, itrial])

                epi_files = []
                for filename in os.listdir(block_dir):
                    if filename.startswith("sw_"):
                        epi_files.append(filename)

                print(block_folder)
                if mode == 'onlyTask':
                    for itrial in range(len(offsets)):
                        t_on = int(onsets_TR[itrial])
                        t_off = int(offsets_TR[itrial])
                        epi_trial = epi_files[t_on:t_off+1]
    #                     print(len(epi_trial))
                        for filename in epi_trial:
                            epis.append(ants.image_read(os.path.join(block_dir, filename)))
                elif mode == 'full':
                    for filename in epi_files:
                            epis.append(ants.image_read(os.path.join(block_dir, filename)))

                # Combinar os EPIs em um único volume
                combined_epi = ants.merge_channels(epis)
#                 print(combined_epi.shape)

                # Salvar o volume combinado como um arquivo .nii.gz
                ants.image_write(combined_epi, output_file)

----->  04_CISC_13392
run: 1
fMRI_block_1_0010
run: 2
fMRI_block_2_0012
run: 3
fMRI_block_3_0016
----->  05_CISC4578
run: 1
fMRI_block_1_0011
run: 2
fMRI_block_2_0013
----->  06_CISC18871
run: 1
fMRI_block_1_0009
run: 2
fMRI_block_2_0011
run: 3
fMRI_block_3_0013
----->  07_CISC18884
run: 1
fMRI_block_1_0010
run: 2
fMRI_block_2_0012
run: 3
fMRI_block_3_0018
----->  10_CISC19050
run: 1
fMRI_block_1_0014
run: 2
fMRI_block_2_0016
run: 3
fMRI_block_3_0018
----->  11_CISC19097
run: 1
fMRI_block_1_0012
run: 2
fMRI_block_2_0014
run: 3
fMRI_block_3_0016
----->  12_CISC19067
run: 2
fMRI_block_2_0014
run: 3
fMRI_block_3_0016
run: 4
fMRI_block_4_0018
----->  13_CISC18321
run: 1
fMRI_block_1_0011
run: 2
fMRI_block_2_0013
run: 3
fMRI_block_3_0015
----->  14_CISC18258
run: 1
fMRI_block_1_0009
run: 2
fMRI_block_2_0011
run: 3
fMRI_block_3_0013
----->  15_CISC18335
run: 1
fMRI_block_1_0010
run: 2
fMRI_block_2_0012
run: 3
fMRI_block_3_0014
----->  16_CISC18191
run: 2
fMRI_block_2_0012
run: 3
fMRI_block_3

In [6]:
import nibabel as nib
img = nib.load(r'C:\Users\erick\Documents\mestrado\subjects\task_volumes\subject_10_run_2_full.nii.gz')
# img = np.squeeze(img.get_fdata())
img.shape

(79, 95, 68, 1, 621)